In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
import graphviz
import matplotlib.pyplot as plt
import seaborn as sns
from ucimlrepo import fetch_ucirepo
from IPython.display import Image, display

# Tải dữ liệu UCI Heart Disease từ ucimlrepo
heart_disease = fetch_ucirepo(id=45)
X = heart_disease.data.features
y = heart_disease.data.targets.squeeze()  # Loại bỏ cột đơn nếu tồn tại

# Kết hợp lại thành một DataFrame đầy đủ
df = pd.concat([X, y.rename("target")], axis=1)

# Lưu ra file CSV
df.to_csv("heart_disease.csv", index=False)

print("Đã lưu dữ liệu thành file heart_disease.csv")
# Chuyển y thành nhị phân: 0 = không bệnh, 1 = có bệnh
y_binary = y.copy()
y_binary[y_binary > 0] = 1

# Hàm vẽ phân bố lớp
def plot_distribution(y_all, y_trains, y_tests, ratios):
    import seaborn as sns
    import matplotlib.pyplot as plt

    fig, axes = plt.subplots(len(ratios)+1, 1, figsize=(6, 4*len(ratios)))

    # Biểu đồ phân phối dữ liệu gốc
    sns.countplot(x=y_all, hue=y_all, ax=axes[0], palette="gray", legend=False)
    axes[0].set_title("Original Dataset")

    for i, ratio in enumerate(ratios):
        # Phân phối tập train
        sns.countplot(x=y_trains[i], hue=y_trains[i], ax=axes[i+1], palette="Blues", legend=False)
        # Phân phối tập test (trùng biểu đồ với train nhưng khác alpha)
        sns.countplot(x=y_tests[i], hue=y_tests[i], ax=axes[i+1], palette="Reds", alpha=0.6, legend=False)

        axes[i+1].set_title(f"Train/Test = {round(ratio*100)}/{round((1-ratio)*100)}")

    plt.tight_layout()
    plt.show()

# Các tỉ lệ chia train/test
ratios = [0.4, 0.6, 0.8, 0.9]

y_trains, y_tests = [], []

for ratio in ratios:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_binary, train_size=ratio, stratify=y_binary, shuffle=True, random_state=42)

    y_trains.append(y_train)
    y_tests.append(y_test)

    # Huấn luyện cây quyết định
    clf = DecisionTreeClassifier(criterion='entropy', random_state=42)
    clf.fit(X_train, y_train)

    # Trực quan hóa cây
    dot_data = export_graphviz(clf, out_file=None,
                                feature_names=X.columns,
                                class_names=['No HD', 'HD'],
                                filled=True, rounded=True,
                                special_characters=True)
    graph = graphviz.Source(dot_data)
    graph.render(f"./output/decision_tree_{int(ratio*100)}", format='png', cleanup=True)
    display(Image(f"./output/decision_tree_{int(ratio*100)}.png"))

# Vẽ phân bố lớp
plot_distribution(y_binary, y_trains, y_tests, ratios)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

for ratio in ratios:
    # Chia lại dữ liệu với y_binary (phân lớp nhị phân)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_binary, train_size=ratio, stratify=y_binary, shuffle=True, random_state=42)

    # Huấn luyện mô hình cây quyết định
    clf = DecisionTreeClassifier(criterion='entropy', random_state=42)
    clf.fit(X_train, y_train)

    # Dự đoán trên tập test
    y_pred = clf.predict(X_test)

    # In báo cáo phân loại
    print(f"\n==== Classification Report (Train/Test = {round(ratio*100)}/{round((1-ratio)*100)}) ====")
    print(classification_report(y_test, y_pred, target_names=['No HD', 'HD']))

    # Tính ma trận nhầm lẫn
    cm = confusion_matrix(y_test, y_pred)

    # Vẽ ma trận nhầm lẫn
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['No HD', 'HD'], yticklabels=['No HD', 'HD'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix (Train/Test = {round(ratio*100)}/{round((1-ratio)*100)})')
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Chia dữ liệu 80/20 với y_binary
X_train, X_test, y_train, y_test = train_test_split(
    X, y_binary, train_size=0.8, stratify=y_binary, shuffle=True, random_state=42)

max_depth_values = [None, 2, 3, 4, 5, 6, 7]
accuracies = []

for depth in max_depth_values:
    clf = DecisionTreeClassifier(criterion='entropy', max_depth=depth, random_state=42)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

    # Trực quan hóa cây
    dot_data = export_graphviz(clf, out_file=None,
                               feature_names=X.columns,
                               class_names=['No HD', 'HD'],
                               filled=True, rounded=True,
                               special_characters=True)
    graph = graphviz.Source(dot_data)
    depth_label = "full" if depth is None else str(depth)
    graph.render(f"./output/decision_tree_maxdepth_{depth_label}", format='png', cleanup=True)
    display(Image(f"./output/decision_tree_maxdepth_{depth_label}.png"))

# In bảng accuracy
print("\nAccuracy scores for different max_depth values:")
print("Max Depth\tAccuracy")
for depth, acc in zip(max_depth_values, accuracies):
    label = "Full Tree" if depth is None else str(depth)
    print(f"{label:<10}\t{acc * 100:.2f}%") 

# Vẽ biểu đồ
labels = ['Full' if d is None else str(d) for d in max_depth_values]

plt.figure(figsize=(8, 5))
# Chuyển accuracies sang phần trăm
accuracies_percent = [acc * 100 for acc in accuracies]
plt.plot(labels, accuracies_percent, marker='o', linestyle='-', color='blue')
plt.ylabel("Accuracy on Test Set (%)")
plt.xlabel("Max Depth")
plt.ylabel("Accuracy on Test Set")
plt.grid(True)
plt.tight_layout()
plt.show()
